In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
import torch.distributed as dist
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
!pip install wandb
import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.


wandb: Currently logged in as: wwblodge1 (use `wandb login --relogin` to force relogin)


True

In [5]:
wandb.init(project="hw9_instance")

wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:
# Assume that this notebook only sees one GPU.
GPU=0

In [7]:
SEED=1

In [8]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [9]:
torch.cuda.device_count()

1

In [10]:
START_EPOCH = 0

In [11]:
ARCH = 'resnet18'
EPOCHS = 1
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH=500
VAL_BATCH=500
WORKERS=2
TRAINDIR="data/train"
VALDIR="data/val"

global_step = 0


In [12]:
TRAINDIR="data/train"
VALDIR="data/val"

In [13]:
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [14]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [15]:
# how many processes in cluster?
WORLD_SIZE = 2
BACKEND = 'nccl'
# where is the master?
# export NCCL_SOCKET_IFNAME=172.31.24.47
URL = 'tcp://172.31.24.176:1234'

In [16]:
#what is my rank?
RANK = 0

In [17]:
dist.init_process_group(backend = BACKEND, init_method= URL,
                                world_size= WORLD_SIZE, rank=RANK)

In [18]:
#torch.cuda.set_device('cpu')
torch.cuda.set_device(GPU)

In [19]:
cudnn.benchmark = True

In [20]:
# def train(train_loader, model, criterion, optimizer, epoch):
#     batch_time = AverageMeter('Time', ':6.3f')
#     data_time = AverageMeter('Data', ':6.3f')
#     losses = AverageMeter('Loss', ':.4e')
#     top1 = AverageMeter('Acc@1', ':6.2f')
#     top5 = AverageMeter('Acc@5', ':6.2f')
#     progress = ProgressMeter(
#         len(train_loader),
#         [batch_time, data_time, losses, top1, top5],
#         prefix="Epoch: [{}]".format(epoch))

#     # switch to train mode
#     model.train()

#     end = time.time()
#     for i, (images, target) in enumerate(train_loader):
#         # measure data loading time
#         data_time.update(time.time() - end)

#         if GPU is not None:
#             images = images.cuda(GPU, non_blocking=True)
#         if torch.cuda.is_available():
#             target = target.cuda(GPU, non_blocking=True)

#         # compute output
#         output = model(images)
#         loss = criterion(output, target)

#         # measure accuracy and record loss
#         acc1, acc5 = accuracy(output, target, topk=(1, 5))
#         losses.update(loss.item(), images.size(0))
#         top1.update(acc1[0], images.size(0))
#         top5.update(acc5[0], images.size(0))

#         # compute gradient and do SGD step
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()

#         if i % PRINT_FREQ == 0:
#             progress.display(i)

def train(train_loader, model, criterion, optimizer, epoch):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [21]:
# def validate(val_loader, model, criterion):
#     batch_time = AverageMeter('Time', ':6.3f')
#     losses = AverageMeter('Loss', ':.4e')
#     top1 = AverageMeter('Acc@1', ':6.2f')
#     top5 = AverageMeter('Acc@5', ':6.2f')
#     progress = ProgressMeter(
#         len(val_loader),
#         [batch_time, losses, top1, top5],
#         prefix='Test: ')

#     # switch to evaluate mode
#     model.eval()

#     with torch.no_grad():
#         end = time.time()
#         for i, (images, target) in enumerate(val_loader):
#             if GPU is not None:
#                 images = images.cuda(GPU, non_blocking=True)
#             if torch.cuda.is_available():
#                 target = target.cuda(GPU, non_blocking=True)

#             # compute output
#             output = model(images)
#             loss = criterion(output, target)

#             # measure accuracy and record loss
#             acc1, acc5 = accuracy(output, target, topk=(1, 5))
#             losses.update(loss.item(), images.size(0))
#             top1.update(acc1[0], images.size(0))
#             top5.update(acc5[0], images.size(0))

#             # measure elapsed time
#             batch_time.update(time.time() - end)
#             end = time.time()

#             if i % PRINT_FREQ == 0:
#                 progress.display(i)

#         # TODO: this should also be done with the ProgressMeter
#         print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
#               .format(top1=top1, top5=top5))

#     return top1.avg

def validate(val_loader, model, criterion):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})

    return top1.avg


In [22]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [23]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [24]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [25]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [26]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [27]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [28]:
normalize = transforms.Normalize(mean=cifar_mean_RGB, std=cifar_std_RGB)

In [29]:
# IMG_SIZE = 32
IMG_SIZE = 224

In [30]:
NUM_CLASSES = 1000

In [31]:
model = models.__dict__[ARCH]()

In [32]:
inf = model.fc.in_features

In [33]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [34]:
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])
# model = torch.nn.parallel.DistributedDataParallel(model)

In [35]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [36]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [37]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [38]:
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
# ])
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [39]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [40]:
# transform_val = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
# ])
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [41]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [42]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [43]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [44]:
best_acc1 = 0

In [45]:
%%time
for epoch in range(START_EPOCH, 2):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})

Epoch: [0][   0/1282]	Time 15.564 (15.564)	Data  5.567 ( 5.567)	Loss 7.0065e+00 (7.0065e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.60 (  0.60)
Epoch: [0][  10/1282]	Time  1.944 ( 2.992)	Data  0.091 ( 1.107)	Loss 6.9282e+00 (6.9693e+00)	Acc@1   0.40 (  0.11)	Acc@5   1.40 (  0.73)
Epoch: [0][  20/1282]	Time  1.097 ( 2.694)	Data  0.002 ( 1.165)	Loss 6.9623e+00 (6.9565e+00)	Acc@1   0.60 (  0.17)	Acc@5   1.20 (  0.90)
Epoch: [0][  30/1282]	Time  0.822 ( 2.624)	Data  0.002 ( 1.337)	Loss 6.8046e+00 (6.9236e+00)	Acc@1   0.60 (  0.24)	Acc@5   1.40 (  1.08)
Epoch: [0][  40/1282]	Time  0.811 ( 2.572)	Data  0.002 ( 1.397)	Loss 6.7398e+00 (6.8891e+00)	Acc@1   0.40 (  0.36)	Acc@5   3.20 (  1.38)
Epoch: [0][  50/1282]	Time  1.526 ( 2.576)	Data  0.003 ( 1.457)	Loss 6.6891e+00 (6.8465e+00)	Acc@1   0.20 (  0.42)	Acc@5   1.40 (  1.60)
Epoch: [0][  60/1282]	Time  1.480 ( 2.563)	Data  0.002 ( 1.431)	Loss 6.5479e+00 (6.8035e+00)	Acc@1   1.20 (  0.48)	Acc@5   3.20 (  1.88)
Epoch: [0][  70/1282]	Time  2.114 ( 2.562

Epoch: [0][ 600/1282]	Time  0.748 ( 2.479)	Data  0.002 ( 0.208)	Loss 4.6287e+00 (5.4434e+00)	Acc@1  10.40 (  6.45)	Acc@5  31.60 ( 17.31)
Epoch: [0][ 610/1282]	Time  0.777 ( 2.480)	Data  0.003 ( 0.205)	Loss 4.5133e+00 (5.4279e+00)	Acc@1  14.20 (  6.57)	Acc@5  31.60 ( 17.56)
Epoch: [0][ 620/1282]	Time  0.767 ( 2.483)	Data  0.002 ( 0.202)	Loss 4.4886e+00 (5.4126e+00)	Acc@1  13.80 (  6.70)	Acc@5  34.20 ( 17.81)
Epoch: [0][ 630/1282]	Time  0.764 ( 2.485)	Data  0.003 ( 0.198)	Loss 4.4832e+00 (5.3972e+00)	Acc@1  14.80 (  6.82)	Acc@5  35.40 ( 18.06)
Epoch: [0][ 640/1282]	Time  0.762 ( 2.486)	Data  0.003 ( 0.195)	Loss 4.4837e+00 (5.3828e+00)	Acc@1  13.40 (  6.93)	Acc@5  34.00 ( 18.30)
Epoch: [0][ 650/1282]	Time  0.741 ( 2.485)	Data  0.003 ( 0.192)	Loss 4.4382e+00 (5.3677e+00)	Acc@1  14.00 (  7.05)	Acc@5  35.40 ( 18.55)
Epoch: [0][ 660/1282]	Time  0.736 ( 2.485)	Data  0.002 ( 0.190)	Loss 4.4752e+00 (5.3526e+00)	Acc@1  12.40 (  7.16)	Acc@5  32.20 ( 18.79)
Epoch: [0][ 670/1282]	Time  0.762 ( 2.486

Epoch: [0][1200/1282]	Time  0.755 ( 2.471)	Data  0.002 ( 0.364)	Loss 3.4975e+00 (4.7250e+00)	Acc@1  28.60 ( 13.06)	Acc@5  53.80 ( 29.50)
Epoch: [0][1210/1282]	Time  0.758 ( 2.471)	Data  0.002 ( 0.375)	Loss 3.5378e+00 (4.7156e+00)	Acc@1  27.40 ( 13.17)	Acc@5  49.60 ( 29.67)
Epoch: [0][1220/1282]	Time  0.755 ( 2.469)	Data  0.002 ( 0.384)	Loss 3.5745e+00 (4.7065e+00)	Acc@1  26.40 ( 13.27)	Acc@5  50.20 ( 29.84)
Epoch: [0][1230/1282]	Time  0.755 ( 2.468)	Data  0.002 ( 0.394)	Loss 3.5085e+00 (4.6970e+00)	Acc@1  26.60 ( 13.37)	Acc@5  51.60 ( 30.01)
Epoch: [0][1240/1282]	Time  0.739 ( 2.468)	Data  0.002 ( 0.404)	Loss 3.6303e+00 (4.6876e+00)	Acc@1  24.60 ( 13.47)	Acc@5  47.00 ( 30.18)
Epoch: [0][1250/1282]	Time  0.735 ( 2.467)	Data  0.002 ( 0.414)	Loss 3.4979e+00 (4.6781e+00)	Acc@1  27.20 ( 13.57)	Acc@5  52.60 ( 30.35)
Epoch: [0][1260/1282]	Time  0.729 ( 2.466)	Data  0.002 ( 0.423)	Loss 3.4259e+00 (4.6690e+00)	Acc@1  25.20 ( 13.66)	Acc@5  53.80 ( 30.51)
Epoch: [0][1270/1282]	Time  0.731 ( 2.466

Epoch: [1][ 500/1282]	Time  5.040 ( 2.444)	Data  4.247 ( 1.532)	Loss 3.6282e+00 (3.5187e+00)	Acc@1  24.60 ( 26.59)	Acc@5  50.00 ( 51.10)
Epoch: [1][ 510/1282]	Time  4.304 ( 2.446)	Data  3.509 ( 1.536)	Loss 3.5319e+00 (3.5188e+00)	Acc@1  26.20 ( 26.58)	Acc@5  51.20 ( 51.09)
Epoch: [1][ 520/1282]	Time  4.336 ( 2.448)	Data  3.542 ( 1.541)	Loss 3.3892e+00 (3.5179e+00)	Acc@1  28.00 ( 26.59)	Acc@5  54.60 ( 51.10)
Epoch: [1][ 530/1282]	Time  3.801 ( 2.446)	Data  2.999 ( 1.541)	Loss 3.4252e+00 (3.5181e+00)	Acc@1  27.80 ( 26.60)	Acc@5  51.60 ( 51.10)
Epoch: [1][ 540/1282]	Time  4.070 ( 2.445)	Data  3.264 ( 1.543)	Loss 3.5008e+00 (3.5177e+00)	Acc@1  25.60 ( 26.59)	Acc@5  51.20 ( 51.10)
Epoch: [1][ 550/1282]	Time  4.198 ( 2.445)	Data  3.407 ( 1.545)	Loss 3.6213e+00 (3.5176e+00)	Acc@1  26.00 ( 26.60)	Acc@5  50.00 ( 51.11)
Epoch: [1][ 560/1282]	Time  4.328 ( 2.446)	Data  3.555 ( 1.549)	Loss 3.5357e+00 (3.5175e+00)	Acc@1  25.60 ( 26.59)	Acc@5  51.00 ( 51.11)
Epoch: [1][ 570/1282]	Time  3.948 ( 2.446

Epoch: [1][1100/1282]	Time  3.568 ( 2.453)	Data  2.803 ( 1.615)	Loss 3.2887e+00 (3.4941e+00)	Acc@1  28.60 ( 26.79)	Acc@5  56.00 ( 51.52)
Epoch: [1][1110/1282]	Time  4.444 ( 2.454)	Data  3.674 ( 1.617)	Loss 3.3266e+00 (3.4929e+00)	Acc@1  28.40 ( 26.81)	Acc@5  52.20 ( 51.55)
Epoch: [1][1120/1282]	Time  4.304 ( 2.453)	Data  3.505 ( 1.617)	Loss 3.4088e+00 (3.4922e+00)	Acc@1  30.40 ( 26.82)	Acc@5  53.80 ( 51.56)
Epoch: [1][1130/1282]	Time  3.987 ( 2.453)	Data  3.186 ( 1.617)	Loss 3.3181e+00 (3.4914e+00)	Acc@1  31.80 ( 26.83)	Acc@5  57.00 ( 51.58)
Epoch: [1][1140/1282]	Time  3.978 ( 2.453)	Data  3.186 ( 1.618)	Loss 3.4190e+00 (3.4906e+00)	Acc@1  25.20 ( 26.85)	Acc@5  53.20 ( 51.59)
Epoch: [1][1150/1282]	Time  4.266 ( 2.453)	Data  3.484 ( 1.619)	Loss 3.4200e+00 (3.4897e+00)	Acc@1  28.80 ( 26.85)	Acc@5  51.80 ( 51.61)
Epoch: [1][1160/1282]	Time  3.981 ( 2.453)	Data  3.195 ( 1.619)	Loss 3.3663e+00 (3.4884e+00)	Acc@1  27.80 ( 26.87)	Acc@5  53.40 ( 51.63)
Epoch: [1][1170/1282]	Time  4.438 ( 2.453

In [46]:

writer.close()
%load_ext tensorboard
%tensorboard --logdir=runs

In [47]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4703), started 0:00:00 ago. (Use '!kill 4703' to kill it.)